In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta

from bulletin.data.notifica import Notifica
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, get_better_notifica
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","notifica")

if not isdir(output):
    makedirs(output)

In [2]:
classificacao_final = ['0','1','2','3','5']

timer = Timer()
notifica = Notifica()
# notifica.read(download_metabase(filename='null.csv',where=f"classificacao_final IS NULL"))
notifica.read(join('input','queries','null.csv'))
for cf in classificacao_final:
    # notifica.read(download_metabase(filename=f"{cf}.csv",where=f"classificacao_final = {cf}"))
    notifica.read(join('input','queries',f"{cf}.csv"))

notifica.update()
print(timer.stop())
notifica.shape()

input\notifica.csv não encontrado, insira ou passe como argumento do método read
reading input\queries\null.csv
reading input\queries\0.csv
reading input\queries\1.csv
reading input\queries\2.csv
reading input\queries\3.csv
reading input\queries\5.csv
normalize and update notifica
checksum não criado
Finished
511.38394951820374


(1717550, 759737, 7077, 949056, 1680)

In [3]:
timer.reset()
casos_confirmados = CasosConfirmados()
casos_confirmados.update()
print(timer.stop())
casos_confirmados.shape()

Casos confirmados excluidos: 0
Obitos confirmados excluidos: 0
166.09018540382385


(376009, 7312)

In [4]:
casosn = notifica.get_casos()
casosc = casos_confirmados.get_casos()
casosc = casosc.loc[casosc['excluir']!='SIM']
casosn.groupby('classificacao_final')[['id']].count()

,id
classificacao_final,
CASO CONFIRMADO,371465
CASO DESCARTADO,569556
CASO SUSPEITO,768802
IGNORADO,700
SINDROME GRIPAL NAO ESPECIFICADA,7027


In [5]:
casos_duplicados = casosn.loc[(casosn['cpf'].notnull()) & (casosn.duplicated('cpf',keep=False))].sort_values('cpf')
print(f"Total de pacientes com mais de uma ocorrencia: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('cpf'))} pacientes pelo CPF que estavam com mais de uma ocorrencia")

all_casos_duplicados = set()
all_duplicados_mantidos = set()
manter = []

for hash, group in casos_duplicados.groupby('cpf'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_cpf.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia: 320093
131695 pacientes pelo CPF que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,8506
CASO DESCARTADO,70046
CASO SUSPEITO,108750
IGNORADO,47
SINDROME GRIPAL NAO ESPECIFICADA,1049


In [6]:
casosn = casosn.drop(index=casos_duplicados)

In [7]:
casos_duplicados = casosn.loc[(casosn['nome_mae'].notnull()) & (casosn.duplicated('hash_mae',keep=False))].sort_values('nome_mae')
# casos_duplicados_nome_mae.loc[casos_duplicados_nome_mae['nome_mae']=='0']

print(f"Total de pacientes com mais de uma ocorrencia pelo nome+nome_mae: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('nome_mae'))} pacientes pelo nome+nome_mae que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('nome_mae'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_mae.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo nome+nome_mae: 55093
23789 pacientes pelo nome+nome_mae que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,1922
CASO DESCARTADO,10886
CASO SUSPEITO,18330
IGNORADO,4
SINDROME GRIPAL NAO ESPECIFICADA,162


In [8]:
casosn = casosn.drop(index=casos_duplicados)

In [9]:
casos_duplicados = casosn.loc[(casosn['hash_idade_resid'].notnull()) & (casosn.duplicated('hash_idade_resid',keep=False))].sort_values('paciente')

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_idade_resid: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_idade_resid'))} pacientes pelo hash_idade_resid que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_idade_resid'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_idade_mun_resid.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo hash_idade_resid: 13278
6546 pacientes pelo hash_idade_resid que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,603
CASO DESCARTADO,1920
CASO SUSPEITO,4182
IGNORADO,3
SINDROME GRIPAL NAO ESPECIFICADA,24


In [10]:
casosn = casosn.drop(index=casos_duplicados)

In [11]:
casos_duplicados = casosn.loc[(casosn['hash_idade_atend'].notnull()) & (casosn.duplicated('hash_idade_atend',keep=False))].sort_values('paciente')
# casos_duplicados_nome_mae.loc[casos_duplicados_nome_mae['nome_mae']=='0']

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_idade_atend: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_idade_atend'))} pacientes pelo hash_idade_atend que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_idade_atend'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_idade_mun_atend.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo hash_idade_atend: 799
399 pacientes pelo hash_idade_atend que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,39
CASO DESCARTADO,93
CASO SUSPEITO,264
IGNORADO,4


In [12]:
casosn = casosn.drop(index=casos_duplicados)

In [13]:
print(f"Das {notifica.shape()[0]} notificações baixadas, {notifica.shape()[0] - casosn.shape[0]} ({len(all_casos_duplicados)}) foram excluidas por serem do mesmo paciente, sendo mantida a que melhor pontuou nos critérios. Sendo assim, o novo total de notificações existente é de {casosn.shape[0]}")
allcasosn = notifica.get_casos()
allcasosn.loc[all_casos_duplicados,'id'].to_csv(join(output,'all_casos_duplicados.csv'), index=False)
allcasosn.loc[all_duplicados_mantidos,'id'].to_csv(join(output,'all_duplicados_mantidos.csv'), index=False)

Das 1717550 notificações baixadas, 226834 (226834) foram excluidas por serem do mesmo paciente, sendo mantida a que melhor pontuou nos critérios. Sendo assim, o novo total de notificações existente é de 1490716


In [14]:
idx_casos_confirmados = casosc[casosc['hash'].isin(casosn['hash_idade_resid'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less'].isin(casosn['hash_idade_resid'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more'].isin(casosn['hash_idade_resid'])].index.tolist()
idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
casos_confirmados_nao_notifica.to_excel(join(output,'casos_confirmados_nao_notifica.xlsx'),index=False)

In [15]:
idx_casos_notifica = casosn[casosn['hash_idade_resid'].isin(casosc['hash'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_idade_resid'].isin(casosc['hash_less'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_idade_resid'].isin(casosc['hash_more'])].index.tolist()
idx_casos_notifica = set(idx_casos_notifica)
casos_confirmados_notifica = casosn.loc[ idx_casos_notifica ].sort_values('id')
casos_confirmados_notifica.to_excel(join(output,'casos_confirmados_notifica.xlsx'),index=False)

In [16]:
notifica_nao_casos_confirmados = casosn.loc[set(casosn.index.tolist()) - set(idx_casos_notifica)]
idx_casos_notifica_idade = notifica_nao_casos_confirmados.loc[notifica_nao_casos_confirmados['hash_idade'].isin(casos_confirmados_nao_notifica['hash_idade'])].index.tolist()
idx_casos_notifica_idade += notifica_nao_casos_confirmados.loc[notifica_nao_casos_confirmados['hash_idade'].isin(casos_confirmados_nao_notifica['hash_idade_less'])].index.tolist()
idx_casos_notifica_idade += notifica_nao_casos_confirmados.loc[notifica_nao_casos_confirmados['hash_idade'].isin(casos_confirmados_nao_notifica['hash_idade_more'])].index.tolist()
print(len(set(idx_casos_notifica_idade)))

7036


In [17]:
notifica_nao_casos_confirmados.loc[notifica_nao_casos_confirmados['cod_classificacao_final'] == 2].shape

(31577, 55)